## LangChain

In [21]:
import os
from dotenv import load_dotenv

In [22]:
load_dotenv()
api = os.getenv("paid_api")

if not api:
    print("OPEN_AI_KEY not found")
else:
    print("API key loaded successfully")

API key loaded successfully


In [23]:
documents = [
    "Python is a high-level programming language known for readability and simplicity.",
    "Machin learning is a subset of AI that enables systems to learn from data.",
    "RAG combines retrieval and generation to provide accurate, grounded response."
]

### Custom Implementation

In [24]:
from sentence_transformers import SentenceTransformer
import numpy as np
from openai import OpenAI


In [25]:

#Generate embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(documents)
# print(doc_embeddings)

'(ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)), '(Request ID: 4854a169-9972-464e-a3e9-a440a04bf1de)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [26]:
#Query and retrieve
query = "What is RAG"
query_embedding = model.encode([query])[0]
# print(query_embedding)

In [27]:
#Compare similarity
similarities = np.dot(doc_embeddings, query_embedding) #uses dot similarities
top_idx = np.argmax(similarities) #returns the indices of the top most similar doc
retrieved_doc = documents[top_idx] #stores the retrieved document using it's index

In [28]:
retrieved_doc

'RAG combines retrieval and generation to provide accurate, grounded response.'

In [32]:
# Generate response
client = OpenAI(api_key=api)
prompt = f"""Context: {retrieved_doc}
Question: {query}
Answer based on the context"""

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

In [33]:
response

ChatCompletion(id='chatcmpl-ClBhqCP7r6nXftTZVHRmBue1qp5Pl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='RAG is a system that combines retrieval and generation techniques to provide accurate and grounded responses.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765362450, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=34, total_tokens=52, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [34]:
print(response.choices[0].message.content)

RAG is a system that combines retrieval and generation techniques to provide accurate and grounded responses.


## LangChain implementation with faiss